In [1]:
from AbstractMesh import AbstractMesh
from Visualization import Viewer
import numpy as np
import utils
from metrics import triangle_aspect_ratio, triangle_area

class Tetmesh(AbstractMesh):
    
    def __init__(self, vertices = None, tets = None, labels = None):
        
        self.tets             = None #npArray (Nx4) 
        self.tet_labels       = None #npArray (Nx1) 
        self.tet2tet          = None #npArray (Nx4?) 
        self.tri2tet          = None #npArray (Nx2?) 
        self.vtx2tet          = None #npArray (NxM)
        
        super(Tetmesh, self).__init__()
        
        if vertices and tets:
            
            self.vertices = np.array(vertices) 
            self.tets = np.array(tets)
            
            if labels:
                self.tet_labels = np.array(labels)
            
            self.__load_operations()
         
    
    # ==================== METHODS ==================== #
    
    def show(self, width = 700, height = 700, mesh_color = None):
        #TODO
        Viewer(self).show(width = width , height = height, mesh_color=mesh_color)
    
    
    @property
    def num_faces(self):
        
        return self.faces.shape[0]

    @property
    def num_tets(self):
        
        return self.tets.shape[0]

    def add_tet(self, tet_id0, tet_id1, tet_id2, tet_id3):
        
        self.add_tets([tet_id0, tet_id1, tet_id2, tet_id3])
        
        
    def add_tets(self, new_tets):
            
        new_tets = np.array(new_tets)
        new_tets.shape = (-1,3)
                
        if new_tets[(new_tets[:,0] > self.num_vertices) | 
                     (new_tets[:,1] > self.num_vertices) | 
                     (new_tets[:,2] > self.num_vertices) | 
                     (new_tets[:,3] > self.num_vertices)].shape[0] > self.num_vertices:
            raise Exception('The ID of a vertex must be less than the number of vertices')

        self.tets = np.concatenate([self.tets, new_tets])
        self.__load_operations()
        
    
    def remove_tet(self, tet_id):
        
        self.remove_tets([tet_id])
        
        
    def remove_tets(self, tet_ids):
        
        tet_ids = np.array(tet_ids)
        mask = np.ones(self.num_tets)
        mask[tet_ids] = 0
        mask = mask.astype(np.bool)
        
        self.tets = self.tets[mask]
        self.__load_operations()
        
    
    def remove_vertex(self,vtx_id):
        
        self.remove_vertices([vtx_id])
    
    
    def remove_vertices(self, vtx_ids):
        
        vtx_ids = np.array(vtx_ids)
        
        for v_id in vtx_ids:
                        
            self.vertices = np.delete(self.vertices, v_id, 0)
            self.tets = self.tets[(self.tets[:,0] != v_id) & 
                                    (self.tets[:,1] != v_id) & 
                                    (self.tets[:,2] != v_id) & 
                                    (self.tets[:,3] != v_id)]
            
            self.tets[(self.tets[:,0] > v_id)] -= np.array([1, 0, 0, 0])
            self.tets[(self.tets[:,1] > v_id)] -= np.array([0, 1, 0, 0])
            self.tets[(self.tets[:,2] > v_id)] -= np.array([0, 0, 1, 0])
            self.tets[(self.tets[:,3] > v_id)] -= np.array([0, 0, 0, 1])
            
            vtx_ids[vtx_ids > v_id] -= 1;
            
        self.__load_operations()
        
        
    def __load_operations(self):
        
        self.__compute_faces()
        self.__compute_adjacencies()
        self._AbstractMesh__update_bounding_box()
        self.set_cut(self.bbox[0,0], self.bbox[1,0], 
                     self.bbox[0,1], self.bbox[1,1], 
                     self.bbox[0,2], self.bbox[1,2])
        #self.__compute_metrics()
    
    def __compute_faces(self):
        self.faces = np.c_[self.tets[:,:3], self.tets[:,1:], self.tets[:, 2:], self.tets[:, 0], self.tets[:, 3], self.tets[:,:2]].reshape(-1,3)
    
    def __compute_adjacencies(self):
        
        map_ = dict()
        adjs = [[] for i in range(self.num_tets)]
        tets_idx = np.repeat(np.array(range(self.num_tets)), 4) 
        
        for f, t in zip(self.faces, tets_idx):

            f1 = (f[0], f[1], f[2])
            f2 = (f[1], f[2], f[0])
            f3 = (f[2], f[0], f[1])

            try:
                tmp = map_[f1]
            except KeyError:
                tmp = None

            if tmp is None:
                map_[(f1[2], f1[1], f1[0])] = t
            else:
                adjs[t].append(map_[f1])
                adjs[map_[f1]].append(t)

            try:
                tmp = map_[f2]
            except KeyError:
                tmp = None

            if tmp is None:
                map_[(f2[0], f2[2], f2[1])] = t
            else:
                adjs[t].append(map_[f2])
                adjs[map_[f2]].append(t)

            try:
                tmp = map_[f3]
            except KeyError:
                tmp = None

            if tmp is None:
                map_[(f3[1], f3[0], f3[2])] = t
            else:
                adjs[t].append(map_[f3])
                adjs[map_[f3]].append(t)
           
        self.tet2tet = np.array([np.array(a) for a in adjs])
  
    
        
    def __compute_face_normals(self):
        
        e1_v = self.vertices[self.faces][:,1] - self.vertices[self.faces][:,0]
        e2_v = self.vertices[self.faces][:,2] - self.vertices[self.faces][:,1]
        
        self.face_normals = np.cross(e1_v, e2_v)
        norm = np.linalg.norm(self.face_normals, axis=1)
        norm.shape = (-1,1)
        self.face_normals = self.face_normals / norm
        
    
    def __compute_vertex_normals(self):
        
        self.vtx_normals = np.array([np.mean(self.face_normals[v2f], axis = 0) for v2f in self.vtx2face])
        norm = np.linalg.norm(self.vtx_normals, axis=1)
        norm.shape = (-1,1)
        self.vtx_normals = self.vtx_normals / norm
        
        
    def load_from_file(self, filename):
        
        ext = filename.split('.')[-1]
        
        if ext == 'mesh':
            self.vertices, self.tets, self.tet_labels = utils.read_mesh(filename)
        else:
            raise Exception("File Extension unknown")
    
    
        self.__load_operations()
        
    
    def save_file(self, filename):
        
        ext = filename.split('.')[-1]
        
        if ext == 'obj':
            utils.save_obj(self, filename)
        
    
    def __compute_metrics(self): 
        
        self.simplex_metrics['area'] = triangle_area(self.vertices, self.faces)
        self.simplex_metrics['aspect_ratio'] = triangle_aspect_ratio(self.vertices, self.faces)
        
    
    def boundary(self, flip_x = False, flip_y = False, flip_z = False):
        
        min_x = self.cut['min_x']
        max_x = self.cut['max_x']
        min_y = self.cut['min_y']
        max_y = self.cut['max_y']
        min_z = self.cut['min_z']
        max_z = self.cut['max_z']
            
        x_range = np.logical_xor(flip_x,((self.simplex_centroids[:,0] >= min_x) & (self.simplex_centroids[:,0] <= max_x)))
        y_range = np.logical_xor(flip_y,((self.simplex_centroids[:,1] >= min_y) & (self.simplex_centroids[:,1] <= max_y)))
        z_range = np.logical_xor(flip_z,((self.simplex_centroids[:,2] >= min_z) & (self.simplex_centroids[:,2] <= max_z)))
        
        return self.faces[ x_range & y_range & z_range]
    
        
    @property
    def face2face(self):
        return self.__face2face

        
    @property
    def simplex_centroids(self):
        
        if self._AbstractMesh__simplex_centroids is None:
            self._AbstractMesh__simplex_centroids = self.vertices[self.faces].mean(axis = 1)
        
        return self._AbstractMesh__simplex_centroids
    
    @property
    def edges(self):
        
        edges =  np.c_[self.faces[:,:2], self.faces[:,1:], self.faces[:,2], self.faces[:,0]]
        edges.shape = (-1,2)
        
        return edges
       
    def __repr__(self):
        self.show()
        return f"Showing {self.boundary().shape[0]} polygons."
    
    
    

In [2]:
a = Tetmesh()

In [4]:
a.load_from_file("human_head.mesh")

IndexError: arrays used as indices must be of integer (or boolean) type